# Generate shape coefficients

The following steps are performed to obtain all the artifacts for shape coefficient use case:

1. Read parameters file
2. Read Mesh description
   * Its path can be specified
   * Or it can be obtained from the simulation configuration file
3. Read data for pressure coefficient

Initialize Path Manager

In [1]:
from cfdmod.use_cases.pressure.path_manager import CePathManager
import pathlib

path_manager = CePathManager(output_path=pathlib.Path("./output/pressure"))

Read post-processing Ce config file

In [2]:
from cfdmod.use_cases.pressure.shape.Ce_config import CeCaseConfig

config_path = pathlib.Path("./fixtures/tests/pressure/Ce_params.yaml")
post_proc_cfg = CeCaseConfig.from_file(config_path)

post_proc_cfg

CeCaseConfig(shape_coefficient={'pattern_1': CeConfig(zoning=ZoningConfig(global_zoning=ZoningModel(x_intervals=[0.0, 25.0, 50.0, 75.0, 100.0, 125.0, 150.0, 175.0, 200.0, 225.0, 250.0], y_intervals=[0.0, 25.0, 50.0, 75.0, 100.0, 125.0, 150.0, 175.0, 200.0], z_intervals=[0.0, 5.0, 10.0, 15.0, 20.0]), no_zoning=['t1_ym'], exclude=['p1_xp'], exceptions={'zoning1': ExceptionZoningModel(x_intervals=[0.0, 25.0, 50.0, 75.0, 100.0, 125.0, 150.0, 175.0, 200.0, 225.0, 250.0], y_intervals=[0.0, 25.0, 50.0, 75.0, 100.0, 125.0, 150.0, 175.0, 200.0], z_intervals=[0.0, 20.0], surfaces=['t2_yp'])}), statistics=['min', 'max', 'std', 'avg', 'skewness', 'kurtosis'], sets={'lanternim': ['L1_xp', 'L2_yp', 'L3_zp_yp', 'L4_zp_ym', 'L5_ym', 'L6_xm']}), 'pattern_2': CeConfig(zoning=ZoningConfig(global_zoning=ZoningModel(x_intervals=[0.0, 250.0], y_intervals=[0.0, 200.0], z_intervals=[0.0, 20.0]), no_zoning=['L2_yp'], exclude=[], exceptions={}), statistics=['min', 'max', 'std', 'avg', 'skewness', 'kurtosis'], s

Normalize LNAS Mesh [Example]

In [3]:
from nassu.lnas import LagrangianFormat

original_mesh_path = pathlib.Path("./fixtures/tests/pressure/galpao/galpao.lnas")
original_mesh = LagrangianFormat.from_file(original_mesh_path)

new_verts = original_mesh.geometry.vertices.copy()

new_verts[:, 0] -= new_verts[:, 0].min()
new_verts[:, 1] -= new_verts[:, 1].min()
new_verts[:, 2] -= new_verts[:, 2].min()

original_mesh.geometry.vertices = new_verts

original_mesh.to_file(original_mesh_path.parent / f"{original_mesh.name}.normalized.lnas")

print(
    original_mesh.name,
    original_mesh.geometry.vertices.shape,
    original_mesh.geometry.triangles.shape,
)

original_mesh.geometry.vertices[:, 0].min(), original_mesh.geometry.vertices[
    :, 1
].min(), original_mesh.geometry.vertices[:, 2].min()

galpao (1549, 3) (2915, 3)


(0.0, 0.0, 0.0)

Read Normalized LNAS Mesh

In [4]:
from nassu.lnas import LagrangianFormat

mesh_path = pathlib.Path("./fixtures/tests/pressure/galpao/galpao.normalized.lnas")
mesh = LagrangianFormat.from_file(mesh_path)

mesh.name, mesh.geometry.vertices.shape, mesh.geometry.triangles.shape

('galpao', (1549, 3), (2915, 3))

Read Pressure Coefficient data

In [5]:
import pandas as pd

cp_data_path = pathlib.Path("./fixtures/tests/pressure/data/cp_t.resampled.hdf")
cp_data = pd.read_hdf(cp_data_path)

cp_data_to_use = cp_data.to_frame() if isinstance(cp_data, pd.Series) else cp_data

Add region index to pressure coefficient data

In [6]:
from cfdmod.api.vtk.write_vtk import write_polydata, merge_polydata
from cfdmod.logger import logger
from cfdmod.use_cases.pressure.shape.Ce_data import process_surface, filter_surface, get_excluded_surfaces


n_timesteps = cp_data_to_use["time_step"].unique().shape[0]
logger.disabled = True
for cfg_label, cfg in post_proc_cfg.shape_coefficient.items():
    processed_polydata = []
    surfaces_to_process = {}
    logger.info(f"Processing {cfg_label} ...")
    for set_lbl in cfg.sets.keys():
        raw_surface = filter_surface(body_mesh=mesh, sfc_label=set_lbl, cfg=cfg, is_set=True)
        surfaces_to_process[set_lbl] = raw_surface

    for sfc in mesh.surfaces.keys():
        if sfc in cfg.surfaces_in_sets:
            continue # Skip surfaces that already are in a set
        if sfc in cfg.zoning.exclude:  # type: ignore (already validated in class)
            logger.info(f"Surface {sfc} ignored!")  # Ignore surface
            continue
        raw_surface = filter_surface(body_mesh=mesh, sfc_label=sfc, cfg=cfg)
        surfaces_to_process[sfc] = raw_surface

    for sfc_lbl, raw_surface in surfaces_to_process.items():
        logger.info(f"Processing surface {sfc_lbl}")
        
        processed_surface = process_surface(
            raw_surface=raw_surface,
            cfg=cfg,
            cp_data=cp_data_to_use,
            n_timesteps=n_timesteps,
        )

        processed_polydata.append(processed_surface.polydata)

        logger.info(f"Processed surface {sfc_lbl}")

    # Output 5: VTK
    merged_polydata = merge_polydata(processed_polydata)
    write_polydata(path_manager.get_vtp_path(mesh.name, cfg_label), merged_polydata)
    
    excluded_sfcs = get_excluded_surfaces(original_mesh=mesh, cfg=cfg)
    if excluded_sfcs:
        excluded_sfcs.export_stl(path_manager.get_excluded_surface_path(cfg_label))

    logger.info(f"Merged polydata for {cfg_label}")

LagrangianGeometry(vertices=array([[ 32.14985  ,  77.800224 ,  16.025024 ],
       [152.73444  ,  99.73702  ,  16.286804 ],
       [ 86.14685  , 158.5      ,   6.8500366],
       ...,
       [126.15463  , 158.5      ,  14.820007 ],
       [ 46.455246 ,  99.64014  ,  16.289246 ],
       [180.75195  ,   8.       ,   9.299988 ]], dtype=float32), triangles=array([[ 688,  537, 1211],
       [ 537, 1043, 1324],
       [1043,  945,  477],
       [ 945,  365,  559],
       [ 276,  734,   38],
       [  24, 1320, 1036],
       [1036, 1517,  703],
       [ 318, 1035,   38],
       [ 365,   38,  559],
       [ 559,  477,  945],
       [ 477, 1324, 1043],
       [1324, 1211,  537],
       [1211,  521,  688],
       [ 521,  831,  532],
       [ 831, 1271, 1085],
       [1085,  532,  831],
       [  24, 1036,  703],
       [ 276,   38,  365],
       [ 734,   24,  703],
       [ 703,  318,  734],
       [ 395,  688,  532],
       [ 688,  521,  532],
       [1035,  559,   38],
       [ 734,  318,   38